In [1]:
import tensorflow as tf # keras 아님 , C언어 같음
import numpy as np
from PIL import Image

In [4]:
# TFLite model을 읽어와서 tensor에 할당하기
interperter = tf.lite.Interpreter(model_path="../Data/CNN/best-gray-cnn-model.tflite")
interperter.allocate_tensors() # 대충 배열 정도

In [5]:
# model의 입력과 출력을 정하기
input_details = interperter.get_input_details()
output_details = interperter.get_output_details()

In [6]:
# model의 input과 output의 구성 출력해보기
print("Input details:")
print(input_details)
print()
print("Output details:")
print(output_details)

Input details:
[{'name': 'serving_default_conv2d_input:0', 'index': 0, 'shape': array([  1, 400, 300,   1], dtype=int32), 'shape_signature': array([ -1, 400, 300,   1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Output details:
[{'name': 'StatefulPartitionedCall:0', 'index': 17, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [7]:
# 예측할 이미지 불러와서 numpy 배열로 변경하기
path = "../Data/FaceGray/Cathy/image_0000.jpg"
load_img = np.array(Image.open(path).resize((400,300)))
input_data = np.array([load_img], dtype = np.float32)
input_data = input_data / 255 # 정규화

In [8]:
# input_detail의 [0]번 값만 출력해보기 <- input_Data의 reshape시 필요
input_details[0] 
# shape 확인
# 이번에 바뀐 내용@ 인터넷 검색해도 안나옴

{'name': 'serving_default_conv2d_input:0',
 'index': 0,
 'shape': array([  1, 400, 300,   1], dtype=int32),
 'shape_signature': array([ -1, 400, 300,   1], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

> 'quantization': (0.0, 0), 
-> 예측력 조절하는 부분. tflite 는 실행할때마다 예측력이 달라진다...?
예측력 조절까지 가능하면 미국 진출 가능 ㄷㄷ

In [9]:
# input_data를 input_details에 넣기
interperter.set_tensor(input_details[0]['index'], input_data.reshape(1,400,300,1) )

In [10]:
# 예측하기
interperter.invoke() # 얘를 실행을 해야 예측하는 것이다@

In [11]:
# 예측한 결과 보기
output_data = interperter.get_tensor(output_details[0]['index'])
print(output_data)

[[3.0428154e-07 2.1834632e-07 9.9999952e-01]]


In [12]:
dirNames = ['Aiden' ,'Andrew', 'Cathy']
print(dirNames[np.argmax(output_data[0])])

Cathy


h5의 핵심만 가져와서 가볍게 사용하는게 TFlite라고 이해하면 된다.              
h5의 학습데이터는 안가져오고 식만 가져오는 정도. 그래서 TFlite는 모든 데이터가 초면이다.                 
학습데이터가 없기때문에 같은 사진을 두번 넣어도 두번다 초면으로 취급한다.                
tflite의 예측률이 낮을 경우 h5의 예측률을 높여서 다시 만들어 본다.               
                       
지금 같은 경우도 h5를 만든 학습 데이터의 양이 적기 때문에 h5에서 데이터는 잘 맞춰도 tflite에서의 예측률은 훨씬 떨어질 수 있는 것이다.                  
--> 그래서 사진 한 장당 -15도 +15도 30개씩으로 늘려서 데이터 양을 18*30 * 3으로 만들어 보면 좋을 듯             
--> + 좌우 반전으로 * 2 로 늘리는 것도 가능 . 예측력 높여서 포트폴리오로 쓰는 것도 가능하다             